# Import libraries

In [1]:
# !pip install git+https://github.com/dubbing-ai/Transliterate.git

In [2]:
import os
from pathlib import Path
import shutil
from tqdm import tqdm
from typing import Tuple
from concurrent.futures import ThreadPoolExecutor
import threading

import sys
sys.path.append('..')
from utils.audio_util import convert_wav_to_flac, resample_audios, trim_silence_with_vad
from utils.file_util import recursive_copy

from transliterate.phonemizer import ThaiPhonemizer

# Moving files to the new directory

In [3]:
thai_phonemizer = ThaiPhonemizer()

In [4]:
!find "../data/raw/TSync2/wav" -type f -name "*.wav" -exec sh -c 'ffmpeg -i "$1" -c:a pcm_mulaw "${1%.wav}.tmp.wav" && mv "${1%.wav}.tmp.wav" "$1"' _ {} \;

File not found - "*.wav"


In [5]:
# Define paths
DEST_DIR = "../data/converted/TSync2-to-vctk-ph"
DEST_TEXT_PATH = os.path.join(DEST_DIR, "txt/TSync2")
DEST_AUDIO_PATH = os.path.join(DEST_DIR, "wav44/TSync2")
SRC_AUDIO_PATH = "../data/raw/TSync2/wav"
SRC_TEXT_PATH = "../data/raw/TSync2/wrd_ph"

# Clean and create directories
if os.path.exists(DEST_DIR):
    print("Clearing destination folder")
    shutil.rmtree(DEST_DIR)
os.makedirs(DEST_TEXT_PATH, exist_ok=True)
os.makedirs(DEST_AUDIO_PATH, exist_ok=True)

# Thread-safe set for character collection
all_chars = set()
chars_lock = threading.Lock()

# Thread-safe list for tracking skipped files
skip_files = []
skip_lock = threading.Lock()

# Thread-safe for phonemizer
phonemizer_lock = threading.Lock()

def process_file_pair(args: Tuple[int, Path, Path]) -> None:
    """Process a single pair of audio and text files"""
    i, audio_file, text_file = args
    try:
        # Process audio
        src_audio = str(audio_file)
        dest_audio = os.path.join(DEST_AUDIO_PATH, f"TSync2_{i:03d}_mic1.flac")
        
        if not convert_wav_to_flac(src_audio, dest_audio):
            raise Exception("Failed to convert audio")
        
        # Process text
        with text_file.open('r', encoding='utf-8') as f:
            with phonemizer_lock:
                clean_text = " ".join(thai_phonemizer.phonemize("".join(f.readline().strip().split("|"))))
            # Thread-safe update of all_chars
            with chars_lock:
                all_chars.update(clean_text)
        
        dest_text = os.path.join(DEST_TEXT_PATH, f"TSync2_{i:03d}.txt")
        with open(dest_text, 'w', encoding='utf-8') as f:
            f.write(clean_text)
            
    except Exception as e:
        print(f"Error processing pair {i}: {e}")
        with skip_lock:
            skip_files.append(i)

# Get sorted lists of files
audio_files = sorted(Path(SRC_AUDIO_PATH).glob("*.wav"))
text_files = sorted(Path(SRC_TEXT_PATH).glob("*.txt"))

# Create processing arguments
process_args = [
    (i, audio_file, text_file) 
    for i, (audio_file, text_file) 
    in enumerate(zip(audio_files, text_files), 1)
]

max_workers = os.cpu_count()

# Process files in parallel with progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    list(tqdm(
        executor.map(process_file_pair, process_args),
        total=len(process_args),
        desc=f"Processing files (using {max_workers} workers)"
    ))

# Print results
print(f"Processed {len(audio_files) - len(skip_files)} file pairs")
print(f"Skipped {len(skip_files)} pairs")
print(f"Unique characters found: {''.join(sorted(all_chars))}")

Clearing destination folder


Processing files (using 16 workers): 100%|██████████| 2710/2710 [14:20<00:00,  3.15it/s]

Processed 2710 file pairs
Skipped 0 pairs
Unique characters found:  2345_abcdefhijklmnoprstuwŋɛɤɯʔʰːᴐ


# Resample, trim, and normalize audio

In [6]:
# Create destination directory if it doesn't exist
os.makedirs("../data/converted/TSync2-to-vctk-ph/wav16_silence_trimmed", exist_ok=True)

# Copy all files from wav32 to wav16_silence_trimmed
src_dir = "../data/converted/TSync2-to-vctk-ph/wav44"
dst_dir = "../data/converted/TSync2-to-vctk-ph/wav16_silence_trimmed"

recursive_copy(src_dir, dst_dir)

In [ ]:
# Resample all files in wav16_silence_trimmed to 16kHz
SAMPLE_RATE = 16000
NUM_RESAMPLE_THREADS = 4

resample_audios(
  input_folders="../data/converted/TSync2-to-vctk-ph/wav16_silence_trimmed",
  file_ext="flac",
  sample_rate=SAMPLE_RATE,
  n_jobs=NUM_RESAMPLE_THREADS
)

Resampling the audio files...
Found 2710 files...


100%|██████████| 2710/2710 [00:17<00:00, 154.76it/s]

Done !


In [8]:
# Trim silence at the beginning and end of each audio file
trim_silence_with_vad(
  input_folder="../data/converted/TSync2-to-vctk-ph/wav16_silence_trimmed",
  file_extension="flac",
)

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\Ming/.cache\torch\hub\master.zip


Found 2710 .flac files to process


Processing files:  51%|█████     | 1376/2710 [06:19<06:18,  3.53it/s]

> The file ..\data\converted\TSync2-to-vctk\wav16_silence_trimmed\TSync2\TSync2_2161_mic1.flac probably does not have speech please check it !!


Processing files:  51%|█████     | 1379/2710 [06:19<04:49,  4.60it/s]

...

> The file ..\data\converted\TSync2-to-vctk\wav16_silence_trimmed\TSync2\TSync2_2674_mic1.flac probably does not have speech please check it !!


Processing files:  72%|███████▏  | 1954/2710 [08:46<01:58,  6.37it/s]

> The file ..\data\converted\TSync2-to-vctk\wav16_silence_trimmed\TSync2\TSync2_2685_mic1.flac probably does not have speech please check it !!


Processing files: 100%|██████████| 2710/2710 [12:48<00:00,  3.52it/s]


Processing complete

Found 58 files with no speech. List saved to ..\data\converted\TSync2-to-vctk\no_speech_files.txt


In [9]:
# Normalize the volume of all audio files to -27dB
!find "../data/converted/TSync2-to-vctk-ph/wav16_silence_trimmed" -type f -name "*.flac" -exec sh -c 'ffmpeg-normalize "$1" -nt rms -t=-27 -o "$1" -ar 16000 -f -ext flac -c:a flac' _ {} \;

File not found - "*.flac"


# Create metadata

In [10]:
DEST_DIR = Path(DEST_DIR)

# Write character files
sorted_chars = sorted(all_chars)
with open(DEST_DIR / 'all_chars_unicode.txt', 'w') as f:
   f.write(''.join(c.encode('unicode_escape').decode('ascii') for c in sorted_chars))
   
with open(DEST_DIR / 'all_chars.txt', 'w') as f:
   f.write(''.join(sorted_chars))